<a href="https://colab.research.google.com/github/nicholaspearson43/Homework-1-Algorithmic-Design/blob/main/Pearson_HW1_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Design, Homework 1
## Nicholas Andrea Pearson - SM3500457

In [26]:
from __future__ import annotations
from numbers import Number
from typing import List, Tuple  
from random import random, seed, randint
from sys import stdout
from timeit import timeit
from math import log, ceil

In [27]:
def gauss_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The values cannot be multiplied')

    result=[[0 for col in range(B.num_of_cols)] for row in range(A.num_of_rows)]

    for row in range(A.num_of_rows):
        for col in range(B.num_of_cols):
            value=0
            for k in range(A.num_of_cols):
                value += A[row][k]*B[k][col]
            result[row][col]=value

    return Matrix(result, clone_matrix=False)

In [28]:
class Matrix(object):
    ''' A simple naive matrix class

    Members
    -------
    _A: List[List[Number]]
        The list of rows that store all the matrix values

    Parameters
    ----------
    A: List[List[Number]]
        The list of rows that store all the matrix values
    clone_matrix: Optional[bool]
        A flag to require a full copy of `A`'s data structure.

    Raises
    ------
    ValueError
        If there are two lists having a different number of values
    '''
    def __init__(self, A: List[List[Number]], clone_matrix: bool = True):
        num_of_cols = None

        for i, row in enumerate(A):
            if num_of_cols is not None:
                if num_of_cols != len(row):
                    raise ValueError('This is not a matrix')
            else:
                num_of_cols = len(row)

        if clone_matrix:
            self._A = [[value for value in row] for row in A]
        else:
            self._A = A

    @property
    def num_of_rows(self) -> int:
        return len(self._A)

    @property
    def num_of_cols(self) -> int:
        if len(self._A) == 0:
            return 0

        return len(self._A[0])

    def copy(self):
        A = [[value for value in row] for row in self._A]

        return Matrix(A, clone_matrix=False)

    def __getitem__(self, y: int):
        ''' Return one of the rows

        Parameters
        ----------
        y: int
            the index of the rows to be returned

        Returns
        -------
        List[Number]
            The `y`-th row of the matrix
        '''
        return self._A[y]

    def __iadd__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] += A[y][x]

        return self

    def __add__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res += A

        return res

    def __isub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] -= A[y][x]

        return self

    def __sub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res -= A

        return res

    def __mul__(self, A: Matrix) -> Matrix:
        ''' Multiply one matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix which multiplies this matrix

        Returns
        -------
        Matrix
            The row-column multiplication between this matrix and that passed
            as parameter

        Raises
        ------
        ValueError
            If the number of columns of this matrix is different from the
            number of rows of `A`
        '''
        return gauss_matrix_mult(self, A)

    def __rmul__(self, value: Number) -> Matrix:
        ''' Multiply one matrix by a numeric value

        Parameters
        ----------
        value: Number
            The numeric value which multiplies this matrix

        Returns
        -------
        Matrix
            The multiplication between `value` and this matrix

        Raises
        ------
        ValueError
            If `value` is not a number
        '''

        if not isinstance(value, Number):
            raise ValueError('{} is not a number'.format(value))

        return Matrix([[value*elem for elem in row] for row in self._A],
                      clone_matrix=False)

    def submatrix(self, from_row: int, num_of_rows: int,
                  from_col: int, num_of_cols: int) -> Matrix:
        ''' Return a submatrix of this matrix

        Parameters
        ----------
        from_row: int
            The first row to be included in the submatrix to be returned
        num_of_rows: int
            The number of rows to be included in the submatrix to be returned
        from_col: int
            The first col to be included in the submatrix to be returned
        num_of_cols: int
            The number of cols to be included in the submatrix to be returned

        Returns
        -------
        Matrix
            A submatrix of this matrix
        '''
        A = [row[from_col:from_col+num_of_cols]
             for row in self._A[from_row:from_row+num_of_rows]]

        return Matrix(A, clone_matrix=False)

    def assign_submatrix(self, from_row: int, from_col: int, A: Matrix):
        for y, row in enumerate(A):
            self_row = self[y + from_row]
            for x, value in enumerate(row):
                self_row[x + from_col] = value

    def __repr__(self):
        return '\n'.join('{}'.format(row) for row in self._A)




class IdentityMatrix(Matrix):
    ''' A class for identity matrices

    Parameters
    ----------
    size: int
        The size of the identity matrix
    '''
    def __init__(self, size: int):
        A = [[1 if x == y else 0 for x in range(size)]
             for y in range(size)]

        super().__init__(A)



# POINT 1
Implement the *strassen_matrix_mult* function to multiply two $2^n \times 2^n$ matrices by using the Strassen algorithm


The function *closest_power* was created to find the closest power of a given number to our value of interest. As it is required that both our matrices are square and with their dimensions being equal and a power of two, we will use this function to verify if these conditions are satisfied. 

In [15]:
def closest_power(base: int, n: int)-> int:
    return ceil(log(n,base))

In [16]:
def get_matrix_quadrants(A: Matrix) -> Tuple[Matrix, Matrix, Matrix, Matrix]:
    A11 = A.submatrix(0, A.num_of_rows // 2, 0, A.num_of_cols // 2)
    A12 = A.submatrix(0, A.num_of_rows // 2, A.num_of_cols // 2, A.num_of_cols // 2)
    A21 = A.submatrix(A.num_of_rows // 2, A.num_of_rows // 2, 0, A.num_of_cols // 2)
    A22 = A.submatrix(A.num_of_rows // 2, A.num_of_rows // 2, A.num_of_cols // 2, A.num_of_cols // 2)

    return A11, A12, A21, A22


In [17]:
def strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:

    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The matrices cannot be multiplied') #check if the two matrices can be multiplied
    
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)

    if A.num_of_cols != A.num_of_rows != B.num_of_cols:
        raise ValueError('At least one of the two matrices is not square') # check if both matrices are square
    
    exp=closest_power(2, A.num_of_cols)
    if A.num_of_cols != 2**exp:
      raise ValueError('The matrices have dimesions that are not a power of two')

    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B)

    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12

    P1 = strassen_matrix_mult(A11, S1)
    P2 = strassen_matrix_mult(S2, B22)
    P3 = strassen_matrix_mult(S3, B11)
    P4 = strassen_matrix_mult(A22, S4)
    P5 = strassen_matrix_mult(S5, S6)
    P6 = strassen_matrix_mult(S7, S8)
    P7 = strassen_matrix_mult(S9, S10)

    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7

    result = Matrix([[0 for x in range(B.num_of_cols)] for y in range(A.num_of_rows)], clone_matrix = False)
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols // 2, C12)
    result.assign_submatrix(result.num_of_rows // 2, 0, C21)
    result.assign_submatrix(result.num_of_rows // 2, result.num_of_cols // 2, C22)

    return result


# POINT 2
Generalize *strassen_matrix_mult* to deal with any kind of matrix pair
that can be multiplied (possibly also non-square matrices) and prove that
the asymptotic complexity does not change.

To generalize the Strassen algorithm for any set of compatible matrices I decided to "pad" our matrices. In this way we can transform even non square matrices into square ones with the required size.  

- The first step is to identify the closest power of two which is greater or equal than the largest value among the number of rows and colums of the two matrices.
- Secondly I created two square matrices with this value as their number of rows and columns. All of the elements of both matrices are initially set to zero.
- I then copied the values our original matrices into the new matrices in the corresponding positions.
- I then used the Strassen Algorithm on these enlarged matrices to compute their product
- Finally I extracted from the result matrix a submatrix with the a number of rows equal to the number of rows of our first factor and a number of columns equal to the number of colums of our second factor.

The matrix we obtain using this procedure is equal to the one we would obtain if we used the Gauss method.

The overall complexity of this method doesn't change from the standard version of Strassen's algorithm and is $\Theta(n^{\log_27})$, where n is the closest power of two which is greater or equal than the largest dimension of both matrices.
If n is the closest power of two which is greater or equal than the maximum dimension we can observe that:
- The cost of creating one of the padded matrices in $n^2$, so the overall cost of this part of the procedure is $2n^2$
- The cost of copying the elements of our original matrices into the padded ones is at most $n^2$ per matrix, so $2n^2$ in total 
- The cost of extractracting the final submatrix in the final step is at most $n^2$
- Finally the cost of performing Strassen's algorithm on our padded matrices is, as stated before, $\Theta(n^{\log_27})$

This leads us to belive that the overall cost of the general version of Strassen's algorithm is $\Theta(n^{\log_27})$



In [19]:
def general_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The values cannot be multiplied')

    m = max(A.num_of_rows,A.num_of_cols, B.num_of_cols)
    n = closest_power(2, m)
    size=2**n
    if size==A.num_of_rows==A.num_of_cols==B.num_of_cols:
        return strassen_matrix_mult(A, B)
    else:
        A_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix=False)
        B_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix=False)
        A_ext.assign_submatrix(0, 0, A)
        B_ext.assign_submatrix(0, 0, B)

        return strassen_matrix_mult(A_ext, B_ext).submatrix(0,A.num_of_rows, 0, B.num_of_cols)


The main disadvantage of this version is that it doesn't consider the size of the matrices we have to multiply. Let's say that matrix $A$ has dimension $2^n \times 2^n$ while matrix $B$ has dimension $2^n \times (2^n+1)$. In this scenario we would have to extend both matrices to size $2^{n+1}\times2^{n+1}$ leading to two matrices which are made of $0$'s for basically $\frac{3}{4}$ of its elements. This would cause a drammattic increase in execution time.

A possible solution for this problem would be to relax the assumption that the dimension must be a power of two and just require it to be even with dimension $m\times2^p$, with $p < n$, where $m$ is a number small enough such that mupliplying two square matrices of dimesion $m$ with the Gauss Method is not computationally intensive. 

In [22]:
def find_close_even(m: int):
    n = closest_power(2, m)
    g = 1
    while g < n:
          for k in range (1, 2**(g+1), 2):
              if k*(2**(n-g))>=m:
                return k, g
          g += 1


In [23]:
def general_strassen_even_matrix_mult(A: Matrix, B: Matrix)-> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The values cannot be multiplied')

    maxi = max(A.num_of_rows,A.num_of_cols, B.num_of_cols)
    n = closest_power(2, maxi)
    size=2**n
    if size==A.num_of_rows==A.num_of_cols==B.num_of_cols:
        return strassen_matrix_mult(A, B)
    else:
        m, i = find_close_even(maxi)
        size=m*2**i
        A_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix=False)
        B_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix=False)
        A_ext.assign_submatrix(0, 0, A)
        B_ext.assign_submatrix(0, 0, B)

        return strassen_matrix_mult(A_ext, B_ext).submatrix(0,A.num_of_rows, 0, B.num_of_cols)

Another technique we could use relaxes the requirements even more and just transforms our matrices into square matrices with an even dimension at every step of the recursive procedure.

We initially search for the largest value among the dimensions of both matrices and check if it is an even or an odd number. If it is even we select this number as the size of the square matrices, otherwise we select this value plus one.  
Performing this procedure recursively we can guarantee that we are able to split our matrices into quadrants until a certain threshold where we use the Gauss Algorithm.

In [ ]:
def general_strassen_square_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The matrices cannot be multiplied')
    maxi = max(A.num_of_cols, A.num_of_rows, B.num_of_cols)

    if maxi < 32:
        return gauss_matrix_mult(A, B)

    if maxi == (maxi//2)*2:
        size=maxi
    else:
        size=maxi+1

    A_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix = False)
    B_ext = Matrix([[0 for x in range(size)] for y in range(size)], clone_matrix = False)
    A_ext.assign_submatrix(0, 0, A)
    B_ext.assign_submatrix(0, 0, B)
    
    A11, A12, A21, A22 = get_matrix_quadrants(A_ext)
    B11, B12, B21, B22 = get_matrix_quadrants(B_ext) 

    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12

    P1 = general_strassen_square_matrix_mult(A11, S1)
    P2 = general_strassen_square_matrix_mult(S2, B22)
    P3 = general_strassen_square_matrix_mult(S3, B11)
    P4 = general_strassen_square_matrix_mult(A22, S4)
    P5 = general_strassen_square_matrix_mult(S5, S6)
    P6 = general_strassen_square_matrix_mult(S7, S8)
    P7 = general_strassen_square_matrix_mult(S9, S10)

    C11 = (P5 + P4 - P2 + P6).submatrix(0, size//2, 0, size//2)
    C12 = (P1 + P2).submatrix(0, size//2, 0, maxi//2)
    C21 = (P3 + P4).submatrix(0, maxi//2, 0, size//2)
    C22 = (P5 + P1 - P3 - P7).submatrix(0, maxi//2, 0, maxi//2)

    result = Matrix([[0 for x in range(maxi)] for y in range(maxi)], clone_matrix = False)
    position=ceil(maxi/ 2)
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, position, C12)
    result.assign_submatrix(position, 0, C21)
    result.assign_submatrix(position, position, C22)

    return result.submatrix(0, A.num_of_rows, 0, B.num_of_cols)

# POINT 3
improve the implementation of the function by reducing the number of
auxiliary matrices and test the effects on the execution time

Solution:

This version of the Strassen algorithm is faster that the standard one as it reuses the same memory region during the recursive steps. 
Each of the $P_j$'s is immediately used after its computation to obtain intermediate results of the quadrants of the output matrix. That same memory region is used immediately after for the following $P_j$.

The time gaines that can be obtained become more significant as the size of the $A$ and $B$ increases.  

Other methods were considered, such as not storing either the $S_i$'s or the $C_k$'s or both but the results were inconsistent in respect to the matrix size  and performed poorly when compared to this method.

This version can be utilized in both "general" versions that were described previously by substituting where required the function.

 

In [24]:
def improved_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:

    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The values cannot be multiplied')

    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)

    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B)

    P = improved_strassen_matrix_mult(A11, B12 - B22) #P1
    C12 = P
    C22 = P
    P = improved_strassen_matrix_mult(A21 + A22, B11) #P3
    C21 = P
    C22 = C22 - P
    P = improved_strassen_matrix_mult(A22, B21 - B11) #P4
    C11 = P
    C21 = C21 + P
    P = improved_strassen_matrix_mult(A11 + A22, B11 + B22) #P5
    C11 = C11 + P
    C22 = C22 + P
    P = improved_strassen_matrix_mult(A12 - A22, B21 + B22) #P6
    C11 = C11 + P
    P = improved_strassen_matrix_mult(A11 - A21, B11 + B12) #P7
    C22 = C22 - P
    P = improved_strassen_matrix_mult(A11 + A12, B22) #P2
    C11 = C11 - P
    C12 = C12 + P

    result = Matrix([[0 for x in range(B.num_of_cols)] for y in range(A.num_of_rows)], clone_matrix = False)
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols // 2, C12)
    result.assign_submatrix(result.num_of_rows // 2, 0, C21)
    result.assign_submatrix(result.num_of_rows // 2, result.num_of_cols // 2, C22)

    return result

## Time comparison

In [25]:
seed(0)

for i in range(12):
    size = 2**i
    stdout.write(f'{size}')

    A = Matrix([[random() for x in range(size)] for y in range(size)])
    B = Matrix([[random() for x in range(size)] for y in range(size)])

    for funct in ['gauss_matrix_mult', 'strassen_matrix_mult', 'improved_strassen_matrix_mult']:
        T = timeit(f'{funct}(A, B)', globals = locals(), number = 1)

        stdout.write('\t{:.3f}'.format(T))
        stdout.flush()

    stdout.write('\n')

1	0.000	0.000	0.000
2	0.000	0.000	0.000
4	0.000	0.000	0.000
8	0.000	0.000	0.000
16	0.003	0.003	0.003
32	0.029	0.018	0.016
64	0.121	0.130	0.131
128	0.873	0.913	0.889
256	6.830	6.378	6.306
512	59.451	45.019	44.797
1024	524.122	317.850	315.974
2048	5052.652	2233.392	2222.870


# POINT 4
Answer the following question: how much is the minimum auxiliary space required to evaluate the Strassen's algorithm. Motivate the answer.


Excluding the space required to store the two input and output matrices (which is $3n^2$), we need another $n^2$ units of memory for our procedure.
The same is true at the second step of our recursive procedure where we need a total of $\frac{3n^2}{4}$ units of memory, and so on. At the generic i-th step we will need to allocate an extra $3\frac{n^2}{2^i}$ units of memory.  
In total the memory required overall will be

$$3n^2+3(\frac n 2)^2+3(\frac n 4)^2 + \dots = 3n^2(1+\frac 1 4 + \frac{1}{4^2} + \dots )\leq 3n^2 (1+\frac{\frac 1 4 }{1-\frac 1 4 })=4n^2$$

These figures are valid if we immediately re-use the memory region freed by the previous computation. For instance, during the computations of $P_j$ we could compute them one and the time, immediately copy the values in the quadrants of interest of the output matrix, and proceed to compute the following temporary matrix in the same memory region, as was done in  point 3. 